In [1]:
import dysregnet
import numpy as np
import pandas as pd
import csv
import math
from itertools import islice
from tqdm import tqdm
from graph_tool.all import *

In [ ]:
# paths for files
meta_path = '../data/TCGA/KIRC/TCGA-KIRC.GDC_phenotype.tsv'
expr_path = '../data/TCGA/KIRC/TCGA-KIRC.htseq_fpkm.tsv'
graph_path = "../data/graph-tool/allCelltypeGraph.gt"
methyl_expr_path = "../data/KIRC/TCGA/TCGA-KIRC.methylation450.tsv"
methyl_path = "../data/TCGA/humanmethylation450_15017482_v1-2.csv"
dysregnet_result_path = "../data/TCGA/dysreg_KIRC.csv"

In [2]:
# get meta data table

meta = pd.read_csv(meta_path, sep='\t', header = 0)
meta["sample_type_id.samples"] = [0 if s==11 else int(s) for s in meta["sample_type_id.samples"]]
meta.drop(meta[(meta["sample_type_id.samples"] != 0) & (meta["sample_type_id.samples"] != 1)].index, inplace=True)

In [3]:
# get gene expression table with samples as rows

expr = pd.read_csv(expr_path, sep='\t', header = None)
expr = expr.T
expr.columns = expr.iloc[0]
expr = expr.tail(-1)
expr.columns = expr.columns.str.split(".").str[0]
cols_expr = expr.columns
expr[cols_expr[1:]] = expr[cols_expr[1:]].apply(pd.to_numeric, errors='coerce')

/tmp/ipykernel_3666209/2466557277.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,2

In [ ]:
# check for amount of control samples

es = set(expr["Ensembl_ID"]) 
ms = set(meta.loc[meta['sample_type_id.samples'] == 0, 'submitter_id.samples'])
es.intersection(ms)

In [4]:
# get a gene set for lookup if gene from graph is in TCGA dataset

gene_list = list(expr.columns.values)
gene_list = gene_list[1:]
gene_set = set(gene_list)

In [5]:
# load graph, get GRN (network), rem position dict, filter genes in expr table

g = load_graph(graph_path)
nodes = []
done_nodes = {}
rem_pos = {}
for e in tqdm(g.edges()):
    rem = g.vp.name[e.source()]
    gene = g.vp.name[e.target()]
    if done_nodes.get(rem) != gene and gene in gene_set:
        nodes.append([rem, gene])
        
        chrom = g.vp.chr[e.source()].split("r")[1]
        if rem_pos.get(chrom) == None:
            rem_pos[chrom] = {}
        rem_pos.get(chrom)[rem] = [g.vp.start[e.source()], g.vp.end[e.source()]]
        done_nodes[rem] = gene
    else: continue
        
grn = pd.DataFrame(nodes, columns=["REM", "GENE"])
s = set(grn["GENE"])
l = list(s)
l.insert(0, 'Ensembl_ID')
expr = expr[l]

2404861it [00:25, 94923.89it/s] 


In [6]:
# get cpg beta values for all cpgs in all samples

cpg = {}
cpg_expr_cols = []                                            
with open(methyl_expr_path) as f_cpg:
    reader = csv.reader(f_cpg, delimiter='\t')
    for l in tqdm(reader):
        if l[0].startswith("Composite"):
            cpg_expr_cols = l[1:]
        else:
            cpg[l[0]] = l[1:]
        

125015it [00:16, 7672.32it/s]


In [7]:
# convert beta values to m values

cpg_m = {}
for c in tqdm(cpg):
    cpg_m[c] = [ math.log2(float(v)/(1-float(v))) if v != "" else "" for v in cpg.get(c)]

100%|█████████████████████████████████████████████████████████████████████████| 125014/125014 [00:27<00:00, 4505.10it/s]


In [ ]:
# get cpg positions for all in same chromosome in dict WITHOUT paying attention to mismatches and mapping from GRCh37 to GRCh38

cpg_pos = {}
counter = 0
with open(methyl_path) as f_cpg_pos:
    reader = csv.reader(f_cpg_pos)
    for l in tqdm(reader):
        if l[0].startswith("cg") or l[0].startswith("ch"):     # rs rausgelassen weil keine Koordinaten
            if cpg_pos.get(l[11]) == None:
                cpg_pos[l[11]] = {}
            cpg_pos.get(l[11])[l[0]] = l[12]
            counter+=1
f_cpg_pos.close()

In [105]:
# bed file with cpgs to map from GRCh37 to GRCh38

f = open("../data/TCGA/cpg_pos_37.bed", "a")
for x, y in cpg_pos.items():
    for v, z in y.items():
        f.write(f"chr{x}:{z}-{z}\t{v}\n")
f.close()

# upload file to LiftOver: https://genome.ucsc.edu/cgi-bin/hgLiftOver

In [8]:
# need to put mismatches from LiftOver manually in mismatch.txt file
# get dict for mismatches with chr and coordinate

mismatches = {}
f_mis = open("../data/TCGA/mismatch.txt", "r")
for l in f_mis.readlines():
    if not l.startswith("#"):
        ls = l.split(":")
        if mismatches.get(ls[0][3:]) == None:
            mismatches[ls[0][3:]] = []
        mismatches.get(ls[0][3:]).append(ls[1].split("-")[0])

In [9]:
# result from LiftOver
# get list with remapped (hg38) coordinates

mapped_list = []
f_mapped = open("../data/TCGA/hglft_genome_22b05_587e30.bed", "r")
for l in f_mapped.readlines():
    pos = l.split(":")[1].split("-")[0]
    mapped_list.append(pos)
f_mapped.close()

In [10]:
# get cpg positions for all in same chromosome in dict WITH attention to mismatches and mapping

cpg_pos = {}
counter = 0
with open(methyl_path) as f_cpg_pos:
    reader = csv.reader(f_cpg_pos)
    for l in tqdm(reader):
        if l[0].startswith("cg") or l[0].startswith("ch"):     # rs rausgelassen weil keine Koordinaten
            if mismatches.get(l[11]) != None and l[12] in  mismatches.get(l[11]):
                continue
            if cpg_pos.get(l[11]) == None:
                cpg_pos[l[11]] = {}
            cpg_pos.get(l[11])[l[0]] = int(mapped_list[counter])
            counter+=1
f_cpg_pos.close()

486436it [00:03, 155894.07it/s]


In [10]:
# map all cpgs to REMs

rem_cpg = {}
for chro, rems in rem_pos.items():
    print(chro)
    loop_start = 0 
    sorted_cpg = dict(sorted(cpg_pos.get(chro).items(), key=lambda x:x[1]))
    keys_list = list(sorted_cpg)
    for r, v in tqdm(rems.items()):
        start = v[0]
        end = v[1]
        rem_cpg[r] = []
        for c, pos in islice(sorted_cpg.items(), loop_start, None):
            if pos > end:
                loop_start = keys_list.index(c)
                break
            if pos >= start and pos <= end:
                rem_cpg.get(r).append(c)

1


100%|█████████████████████████████████████████████████████████████████████████| 127434/127434 [01:59<00:00, 1062.61it/s]


10


100%|███████████████████████████████████████████████████████████████████████████| 65348/65348 [00:24<00:00, 2679.17it/s]


11


...

100%|██████████████████████████████████████████████████████████████████████████| 62408/62408 [00:05<00:00, 10821.28it/s]


X


100%|███████████████████████████████████████████████████████████████████████████| 57857/57857 [00:08<00:00, 6963.35it/s]


Y


100%|███████████████████████████████████████████████████████████████████████████| 1489/1489 [00:00<00:00, 136551.48it/s]


In [ ]:
# kick out REMs with no mapped cpgs and print to file
rc_map = open("../data/TCGA/rem_cpg_map", "w")
rem_filt = dict()
for r, v in rem_cpg.items():
    if len(v) != 0:
        rem_filt[r] = v
        cpgs = ",".join(v)
        rc_map.write(f"{r} {cpgs}\n")
rc_map.close()

In [12]:
# open file with cpgs mapped to rem and get filtered rem dict
rc_map = open("../data/TCGA/rem_cpg_map", "r")
rem_filt = dict()
for line in rc_map.readlines():
    l = line.split(" ")
    rem_filt[l[0]] = l[1].replace("\n", "").split(",")
        

In [13]:
# for each REM get mean val over all contained cpgs => this for all samples 
rem_means = {}
for r, c in tqdm(rem_filt.items()):
    lists = [] 
    for l in c:
        if cpg.get(l) != None and "" not in cpg.get(l):
            lists.append(list(map(float, cpg.get(l))))
    if len(lists) > 1:
        arrays = [np.array(x) for x in lists]
        means = [np.mean(k) for k in zip(*arrays)]
    elif len(lists) == 1:
        means = lists[0]
    else:
        continue
    rem_means[r] = means

100%|███████████████████████████████████████████████████████████████████████████| 97119/97119 [00:16<00:00, 5872.92it/s]


In [14]:
# create dataframe from rem_means

rem_df = pd.DataFrame.from_dict(rem_means)
rem_df.insert(0, "Ensembl_ID", cpg_expr_cols)
# dataf = dataf.set_axis(cpg_expr_cols, axis=0)

In [15]:
# filter expression table

samples_final = set(cpg_expr_cols).intersection(set(expr["Ensembl_ID"]))
rem_df_final = rem_df.loc[rem_df['Ensembl_ID'].isin(samples_final)]
expr_final = expr.loc[expr['Ensembl_ID'].isin(samples_final)]
expr_all_final = expr_final.join(rem_df_final.set_index('Ensembl_ID'), on='Ensembl_ID')
expr_all_final.reset_index(drop=True, inplace=True)
expr_all_final = expr_all_final.loc[:, ~(expr_all_final == 0.0).any()]

In [16]:
# get all other final dataframes for dyregnet call

rem_final = list(rem_df_final.columns)[1:]
genes_final = list(expr_final.columns)[1:]

meta_final = meta.loc[meta["submitter_id.samples"].isin(samples_final)]
meta_final.reset_index(drop=True, inplace=True)

grn_final = grn.loc[grn["REM"].isin(rem_final)]
grn_final = grn.loc[grn["GENE"].isin(genes_final)]
grn_final.reset_index(drop=True, inplace=True)

In [17]:
# check if enough samples contained for dysregnet (at least 8)

list(meta_final["sample_type_id.samples"]).count(0)

24

In [18]:
data=dysregnet.run(expression_data=expr_all_final,
                   meta=meta_final, 
                   CatCov=["race.demographic", "gender.demographic"],
                   ConCov=["days_to_birth.demographic"],
                   GRN=grn_final,
                   normaltest=True,
                   R2_threshold=.2,
                   conCol='sample_type_id.samples')

35472it [01:41, 349.19it/s]


In [30]:
# filter dysregnet results

res = data.get_results()
sample_list = res.iloc[:,0].tolist()
res = res[res.columns[1:]]
res_filt = res.loc[:, (res > 0.0).any()]
res_filt.index = sample_list

True

In [35]:
# save results as csv

res_filt.to_csv(dysregnet_result_path, sep="\t")